In [1]:
import torch
from torch import nn, FloatTensor
from tools.DARNN import DARNN
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
from numpy import concatenate as np_cat
from sklearn.preprocessing import MinMaxScaler

from tqdm import trange
from tools.DarnnTrainingTools import Trainer
device = torch.device('cuda')

In [2]:
data = pd.read_parquet('data/all_data.parquet', engine='pyarrow')
temp = pd.read_csv('data/weather_actual.csv')
data = pd.concat([data, temp.iloc[:, 1:]], axis=1)
del temp

data = data.drop(columns=['time']).values

In [3]:
class CustomDataset(Dataset):
    def __init__(self, data: np.ndarray, T=72, P=48):
        super().__init__()

        self.x1 = []; x1_append = self.x1.append
        self.x2 = []; x2_append = self.x2.append
        self.y = []; y_append = self.y.append
        self.length = data.shape[0] - T - P + 1

        for i in trange(self.length):
            x1 = data[i:i+T, -13:]  # actual weather
            x1 = np_cat([x1, data[i+T:i+T+P, 1:14]], axis=0)  # actual & forecasted weather
            x1 = np_cat([x1, data[i:i+T+P, 14:19]], axis=1)  # forecasted gens
            x1_append(FloatTensor(x1))
            
            x2_append(FloatTensor(data[i:i+T, 0].reshape(T, 1)))
            y_append(FloatTensor(data[i+T:i+T+P, 0]))


    def __len__(self):
        return self.length


    def __getitem__(self, index):
        return self.x1[index], self.x2[index], self.y[index]

In [4]:
train_set = CustomDataset(data[:24*400])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)

val_set = CustomDataset(data[24*400:24*450])
val_loader = DataLoader(train_set, batch_size=512, shuffle=False)

test_set = CustomDataset(data[24*450:24*484])
test_loader = DataLoader(test_set, batch_size=512, shuffle=False)

100%|██████████| 697/697 [00:00<00:00, 8628.73it/s]


In [5]:
criterion = nn.MSELoss()
model = DARNN(72, 48, 18, 32, 32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

trainer = Trainer(criterion, device, 'checkpoints/darnn_best.pt')

In [6]:
best_model = trainer.train(model, optimizer, train_loader, val_loader, patience=10, epochs=100)

Epoch 43: 100%|██████████| 297/297 [01:11<00:00,  4.18it/s, Train Loss=76.2054, Valid Loss=75.4781, Valid R2=0.8075, Valid MAE=4.8705, Valid Profit=-763.0]

Early Stopped


In [7]:
trainer.test(test_loader)

Test Loss: 133.8098 | Test R2: 0.5011 | Test MAE: 6.1811 | Test Profit: -659.5
